# D. Langkah Praktikum
Praktikum ini akan memandu Anda membuat aplikasi pencatat pengeluaran harian dari awal hingga akhir, menggunakan struktur modular, kelas OOP, database SQLite, dan antarmuka web Streamlit. 
Penjelasan Kelas Diagram (Diagram Komponen): 
Diagram ini menunjukkan struktur akhir aplikasi yang modular: 
•	streamlit_app.py: Lapisan presentasi (UI) yang menggunakan Streamlit dan memanggil lapisan logika bisnis. 
•	manajer_anggaran.py: Lapisan logika bisnis/repository, menggunakan model dan utilitas database. 
•	model.py: Lapisan data, mendefinisikan struktur Transaksi. 
•	database.py: Lapisan akses data, menangani interaksi SQLite. 
•	konfigurasi.py: Menyimpan konstanta. 
•	pengeluaran_harian.db: Penyimpanan data fisik. 
•	External Libs: Library yang digunakan. 
Panah menunjukkan dependensi antar modul. Ini adalah arsitektur yang umum digunakan untuk memisahkan tanggung jawab dalam sebuah aplikasi. 

## Langkah 1: Persiapan Awal (Konfigurasi & Setup Database) 
Tujuan: Menyiapkan file konfigurasi dasar dan skrip untuk membuat database SQLite beserta tabelnya. 
 


### file konfigurasi.py

In [ ]:
# konfigurasi.py 
import os 
  
BASE_DIR = os.path.dirname(os.path.abspath(__file__)) 
NAMA_DB = 'pengeluaran_harian.db' 
DB_PATH = os.path.join(BASE_DIR, NAMA_DB) 
KATEGORI_PENGELUARAN = ["Makanan", "Transportasi", "Hiburan", "Tagihan", 
"Belanja", "Kesehatan", "Pendidikan", "Lainnya"] 
KATEGORI_DEFAULT = "Lainnya" 

### File 2: setup_db_pengeluaran.py 
Buat file ini untuk setup database awal. Jalankan file ini sekali saja dari terminal (python setup_db_pengeluaran.py). 


In [ ]:
# setup_db.py
import sqlite3
from konfigurasi import DB_PATH

def setup_database():
    """Membuat database dan tabel pekerjaan jika belum ada."""
    print(f"Membuat database di: {DB_PATH}")
    try:
        with sqlite3.connect(DB_PATH) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS pekerjaan (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    nama_perusahaan TEXT NOT NULL,
                    latitude REAL NOT NULL,
                    longitude REAL NOT NULL
                );
            """)
            print("Tabel 'pekerjaan' berhasil dibuat atau sudah ada.")
    except sqlite3.Error as e:
        print(f"Error saat membuat database: {e}")

if __name__ == "__main__":
    setup_database()

Observasi: Setelah menjalankan setup_db_pengeluaran.py, Anda akan memiliki file database pengeluaran_harian.db dengan tabel transaksi di direktori proyek Anda. 

## Langkah 2: Modul Akses Database (database.py) 
Tujuan: Membuat modul terpisah yang berisi fungsi-fungsi untuk berinteraksi dengan database SQLite (koneksi, eksekusi query, fetch data). 


In [ ]:
# database.py
import sqlite3
import pandas as pd
from konfigurasi import DB_PATH

def get_db_connection():
    """Membuat dan mengembalikan koneksi ke database."""
    try:
        conn = sqlite3.connect(DB_PATH, timeout=10)
        conn.row_factory = sqlite3.Row
        return conn
    except sqlite3.Error as e:
        print(f"Error koneksi database: {e}")
        return None

def execute_query(query, params=()):
    """Menjalankan query INSERT/UPDATE/DELETE."""
    with get_db_connection() as conn:
        if conn:
            try:
                cursor = conn.cursor()
                cursor.execute(query, params)
                conn.commit()
                return cursor.lastrowid
            except sqlite3.Error as e:
                print(f"Query gagal: {e}")
                return None

def get_dataframe(query, params=()):
    """Mengambil data dari database dan mengembalikannya sebagai DataFrame."""
    with get_db_connection() as conn:
        if conn:
            try:
                return pd.read_sql_query(query, conn, params=params)
            except Exception as e:
                print(f"Gagal mengambil DataFrame: {e}")
                return pd.DataFrame()
    return pd.DataFrame()

Observasi: Modul ini berisi fungsi-fungsi inti untuk semua operasi database. Fungsi setup_database_initial juga disertakan di sini agar bisa dipanggil oleh modul lain jika diperlukan (misalnya, saat AnggaranHarian pertama kali dibuat). 

## Langkah 3: Modul Model Data (model.py) 
Tujuan: Mendefinisikan kelas Transaksi yang merepresentasikan struktur data untuk satu record pengeluaran. 

In [1]:
# model.py

import datetime
import locale


class Transaksi:
    """Merepresentasikan satu entitas transaksi pengeluaran (Data Class)."""

    def __init__(self, deskripsi: str, jumlah: float, kategori: str, tanggal: datetime.date | str, id_transaksi: int | None = None):
        self.id = id_transaksi
        self.deskripsi = str(deskripsi) if deskripsi else "Tanpa Deskripsi"

        # Validasi jumlah
        try:
            jumlah_float = float(jumlah)
            if jumlah_float > 0:
                self.jumlah = jumlah_float
            else:
                self.jumlah = 0.0
                print(f"Peringatan: Jumlah '{jumlah}' harus positif.")
        except (ValueError, TypeError):
            self.jumlah = 0.0
            print(f"Peringatan: Jumlah '{jumlah}' tidak valid.")

        # Validasi kategori
        self.kategori = str(kategori) if kategori else "Lainnya"

        # Validasi tanggal
        if isinstance(tanggal, datetime.date):
            self.tanggal = tanggal
        elif isinstance(tanggal, str):
            try:
                self.tanggal = datetime.datetime.strptime(tanggal, "%Y-%m-%d").date()
            except ValueError:
                self.tanggal = datetime.date.today()
                print(f"Peringatan: Format tanggal '{tanggal}' salah.")
        else:
            self.tanggal = datetime.date.today()
            print(f"Peringatan: Tipe tanggal '{type(tanggal)}' tidak valid.")

    def __repr__(self) -> str:
        try:
            locale.setlocale(locale.LC_ALL, 'id_ID.UTF8')
            jml_str = locale.format_string("%.0f", self.jumlah, grouping=True)
        except:
            jml_str = f"{self.jumlah:.0f}"

        return (
            f"Transaksi(ID:{self.id}, Tgl:{self.tanggal.strftime('%Y-%m-%d')}, "
            f"Jml:{jml_str}, Kat:'{self.kategori}', Desc:'{self.deskripsi}')"
        )

    def to_dict(self) -> dict:
        return {
            "deskripsi": self.deskripsi,
            "jumlah": self.jumlah,
            "kategori": self.kategori,
            "tanggal": self.tanggal.strftime("%Y-%m-%d")
        }


Observasi: Kelas ini fokus pada struktur data dan validasi dasar saat objek dibuat. 

## Langkah 4: Modul Manajer Anggaran (manajer_anggaran.py) 
Tujuan: Membuat kelas AnggaranHarian yang berisi logika bisnis aplikasi dan menggunakan modul database.py untuk akses data. 

In [ ]:
# manajer_anggaran.py
import datetime
import pandas as pd
from model import Transaksi
import database  # Impor modul database kita

class AnggaranHarian:
    """Mengelola logika bisnis pengeluaran harian (Repository Pattern)."""
    _db_setup_done = False  # Flag untuk memastikan setup DB hanya dicek sekali per sesi

    def __init__(self):
        if not AnggaranHarian._db_setup_done:
            print("[AnggaranHarian] Melakukan pengecekan/setup database awal...")
            if database.setup_database_initial():  # Panggil fungsi setup dari database.py
                AnggaranHarian._db_setup_done = True
                print("[AnggaranHarian] Database siap.")
            else:
                print("[AnggaranHarian] KRITICAL: Setup database awal GAGAL!")

    def tambah_transaksi(self, transaksi: Transaksi) -> bool:
        if not isinstance(transaksi, Transaksi) or transaksi.jumlah <= 0:
            return False
        sql = "INSERT INTO transaksi (deskripsi, jumlah, kategori, tanggal) VALUES (?, ?, ?, ?)"
        params = (transaksi.deskripsi, transaksi.jumlah, transaksi.kategori, transaksi.tanggal.strftime("%Y-%m-%d"))
        last_id = database.execute_query(sql, params)
        if last_id is not None:
            transaksi.id = last_id
            return True
        return False

    def get_semua_transaksi_obj(self) -> list[Transaksi]:
        sql = "SELECT id, deskripsi, jumlah, kategori, tanggal FROM transaksi ORDER BY tanggal DESC, id DESC"
        rows = database.fetch_query(sql, fetch_all=True)
        transaksi_list = []
        if rows:
            for row in rows:
                transaksi_list.append(
                    Transaksi(
                        id_transaksi=row['id'],
                        deskripsi=row['deskripsi'],
                        jumlah=row['jumlah'],
                        kategori=row['kategori'],
                        tanggal=row['tanggal']
                    )
                )
        return transaksi_list

    def get_dataframe_transaksi(self, filter_tanggal: datetime.date | None = None) -> pd.DataFrame:
        query = "SELECT tanggal, kategori, deskripsi, jumlah FROM transaksi"
        params = None
        if filter_tanggal:
            query += " WHERE tanggal = ?"
            params = (filter_tanggal.strftime("%Y-%m-%d"),)
        query += " ORDER BY tanggal DESC, id DESC"
        df = database.get_dataframe(query, params=params)
        if not df.empty:
            try:
                import locale
                locale.setlocale(locale.LC_ALL, 'id_ID.UTF-8')
                df['Jumlah (Rp)'] = df['jumlah'].map(lambda x: locale.currency(x or 0, grouping=True, symbol='Rp ')[:-3])
            except:
                df['Jumlah (Rp)'] = df['jumlah'].map(lambda x: f"Rp {x or 0:,.0f}".replace(",", "."))
            df = df[['tanggal', 'kategori', 'deskripsi', 'Jumlah (Rp)']]
        return df

    def hitung_total_pengeluaran(self, tanggal: datetime.date | None = None) -> float:
        sql = "SELECT SUM(jumlah) FROM transaksi"
        params = None
        if tanggal:
            sql += " WHERE tanggal = ?"
            params = (tanggal.strftime("%Y-%m-%d"),)
        result = database.fetch_query(sql, params=params, fetch_all=False)
        if result and result[0] is not None:
            return float(result[0])
        return 0.0

    def get_pengeluaran_per_kategori(self, tanggal: datetime.date | None = None) -> dict:
        hasil = {}
        sql = "SELECT kategori, SUM(jumlah) FROM transaksi"
        params = []
        if tanggal:
            sql += " WHERE tanggal = ?"
            params.append(tanggal.strftime("%Y-%m-%d"))
        sql += " GROUP BY kategori HAVING SUM(jumlah) > 0 ORDER BY SUM(jumlah) DESC"
        rows = database.fetch_query(sql, params=tuple(params) if params else None, fetch_all=True)
        if rows:
            for row in rows:
                kategori = row['kategori'] if row['kategori'] else "Lainnya"
                jumlah = float(row[1]) if row[1] is not None else 0.0
                hasil[kategori] = jumlah
        return hasil

Observasi: Kelas ini mengimpor Transaksi dari model.py dan semua fungsi dari 
database.py. __init__-nya memanggil database.setup_database_initial() untuk memastikan tabel siap. Metode-metode lain fokus pada logika bisnis dan mendelegasikan operasi DB ke modul database. 

## Langkah 5: Aplikasi Utama Streamlit (main_app.py) 
Tujuan: Membuat antarmuka pengguna web interaktif menggunakan Streamlit yang mengintegrasikan semua komponen backend. 

In [ ]:
# main_app.py
import streamlit as st
import datetime
import pandas as pd
import locale

try:
    locale.setlocale(locale.LC_ALL, 'id_ID.UTF-8')
except locale.Error:
    try:
        locale.setlocale(locale.LC_ALL, 'Indonesian_Indonesia.1252')
    except:
        print("Locale id_ID/Indonesian tidak tersedia.")

def format_rp(angka):
    try:
        return locale.currency(angka or 0, grouping=True, symbol='Rp ')[:-3]
    except:
        return f"Rp {angka or 0:,.0f}".replace(",", ".")

try:
    from model import Transaksi
    from manajer_anggaran import AnggaranHarian
    from konfigurasi import KATEGORI_PENGELUARAN  # Ambil list kategori
except ImportError as e:
    st.error(f"Gagal mengimpor modul: {e}. Pastikan file .py lain ada.")
    st.stop()

st.set_page_config(page_title="Catatan Pengeluaran", layout="wide", initial_sidebar_state="expanded")

# --- Inisialisasi Pengelola Anggaran (Gunakan Cache) ---
@st.cache_resource
def get_anggaran_manager():
    print(">>> STREAMLIT: (Cache Resource) Menginisialisasi AnggaranHarian...")
    return AnggaranHarian()  # Ini akan memicu cek DB/Tabel di __init__

anggaran = get_anggaran_manager()

# --- Fungsi Halaman/UI ---
def halaman_input(anggaran: AnggaranHarian):
    st.header("  Tambah Pengeluaran Baru")
    with st.form("form_transaksi_baru", clear_on_submit=True):
        col1, col2 = st.columns([3, 1])
        with col1:
            deskripsi = st.text_input("Deskripsi*", placeholder="Contoh: Makan siang")
        with col2:
            kategori = st.selectbox("Kategori*:", KATEGORI_PENGELUARAN, index=0)
        col3, col4 = st.columns([1, 1])
        with col3:
            jumlah = st.number_input("Jumlah (Rp)*:", min_value=0.01, step=1000.0, format="%.0f", value=None, placeholder="Contoh: 25000")
        with col4:
            tanggal = st.date_input("Tanggal*:", value=datetime.date.today())
        submitted = st.form_submit_button("  Simpan Transaksi")
        if submitted:
            if not deskripsi:
                st.warning("Deskripsi wajib!", icon=" ")
            elif jumlah is None or jumlah <= 0:
                st.warning("Jumlah wajib!", icon=" ")
            else:
                with st.spinner("Menyimpan..."):
                    tx = Transaksi(deskripsi, float(jumlah), kategori, tanggal)
                    if anggaran.tambah_transaksi(tx):
                        st.success(f"OK! Simpan.", icon=" ")
                        st.cache_data.clear()
                        st.rerun()
                    else:
                        st.error("Gagal simpan.", icon=" ")

def halaman_riwayat(anggaran: AnggaranHarian):
    st.subheader("Detail Semua Transaksi")
    if st.button("  Refresh Riwayat"):
        st.cache_data.clear()
        st.rerun()
    with st.spinner("Memuat riwayat..."):
        df_transaksi = anggaran.get_dataframe_transaksi()
    if df_transaksi is None:
        st.error("Gagal ambil riwayat.")
    elif df_transaksi.empty:
        st.info("Belum ada transaksi.")
    else:
        st.dataframe(df_transaksi, use_container_width=True, hide_index=True)

def halaman_ringkasan(anggaran: AnggaranHarian):
    st.subheader("Ringkasan Pengeluaran")
    col_filter1, col_filter2 = st.columns([1, 2])
    with col_filter1:
        pilihan_periode = st.selectbox("Filter Periode:", ["Semua Waktu", "Hari Ini", "Pilih Tanggal"], key="filter_periode", on_change=lambda: st.cache_data.clear())

    tanggal_filter = None
    label_periode = "(Semua Waktu)"
    if pilihan_periode == "Hari Ini":
        tanggal_filter = datetime.date.today()
        label_periode = f"({tanggal_filter.strftime('%d %b')})"
    elif pilihan_periode == "Pilih Tanggal Tertentu":
        if 'tanggal_pilihan_state' not in st.session_state:
            st.session_state.tanggal_pilihan_state = datetime.date.today()
        tanggal_filter = st.date_input(
            "Pilih Tanggal:",
            value=st.session_state.tanggal_pilihan_state,
            key="tanggal_pilihan",
            on_change=lambda: setattr(st.session_state, 'tanggal_pilihan_state', st.session_state.tanggal_pilihan) or st.cache_data.clear()
        )
        label_periode = f"({tanggal_filter.strftime('%d %b %Y')})"

    with col_filter2:
        @st.cache_data(ttl=300)  # Cache hasil total
        def hitung_total_cached(tgl_filter):
            return anggaran.hitung_total_pengeluaran(tanggal=tgl_filter)
        total_pengeluaran = hitung_total_cached(tanggal_filter)
        st.metric(label=f"Total Pengeluaran {label_periode}", value=format_rp(total_pengeluaran))

    st.divider()
    st.subheader(f"Pengeluaran per Kategori {label_periode}")

    @st.cache_data(ttl=300)  # Cache hasil kategori
    def get_kategori_cached(tgl_filter):
        return anggaran.get_pengeluaran_per_kategori(tanggal=tgl_filter)

    with st.spinner(f"Memuat ringkasan kategori..."):
        dict_per_kategori = get_kategori_cached(tanggal_filter)

    if not dict_per_kategori:
        st.info(f"Tidak ada data untuk periode ini.")
    else:
        try:
            data_kategori = [{"Kategori": kat, "Total": jml} for kat, jml in dict_per_kategori.items()]
            df_kategori = pd.DataFrame(data_kategori).sort_values(by="Total", ascending=False).reset_index(drop=True)
            df_kategori['Total (Rp)'] = df_kategori['Total'].apply(format_rp)
            col_kat1, col_kat2 = st.columns(2)
            with col_kat1:
                st.write("Tabel:")
                st.dataframe(df_kategori[['Kategori', 'Total (Rp)']], hide_index=True, use_container_width=True)
            with col_kat2:
                st.write("Grafik:")
                st.bar_chart(df_kategori.set_index('Kategori')['Total'], use_container_width=True)
        except Exception as e:
            st.error(f"Gagal tampilkan ringkasan: {e}")

# --- Fungsi Utama Aplikasi Streamlit ---
def main():
    st.sidebar.title("  Catatan Pengeluaran")
    menu_pilihan = st.sidebar.radio("Pilih Menu:", ["Tambah", "Riwayat", "Ringkasan"], key="menu_utama")
    st.sidebar.markdown("---")
    st.sidebar.info("Jobsheet - Aplikasi Keuangan")
    manajer_anggaran = get_anggaran_manager()
    if menu_pilihan == "Tambah":
        halaman_input(manajer_anggaran)
    elif menu_pilihan == "Riwayat":
        halaman_riwayat(manajer_anggaran)
    elif menu_pilihan == "Ringkasan":
        halaman_ringkasan(manajer_anggaran)
    st.markdown("---")
    st.caption("Pengembangan Aplikasi Berbasis OOP")

if __name__ == "__main__":
    main()  # Jalankan fungsi utama

Observasi:  
•	File ini sekarang menjadi inti dari aplikasi, mengimpor kelas dan fungsi dari modul lain. 
•	@st.cache_resource digunakan untuk AnggaranHarian agar tidak selalu membuat koneksi DB baru. @st.cache_data digunakan pada fungsi yang mengambil data dari AnggaranHarian untuk ditampilkan di UI, agar query DB tidak dijalankan terus menerus jika filter tidak berubah. 
•	UI dibagi menjadi beberapa fungsi halaman (halaman_input, halaman_riwayat, halaman_ringkasan). 
•	Navigasi menggunakan st.sidebar.radio. 
•	Setiap halaman memanggil metode yang sesuai dari instance AnggaranHarian untuk mendapatkan atau menyimpan data. 

## Langkah 6: Menjalankan dan Menguji Aplikasi Modular 
Tujuan: Memastikan semua modul bekerja sama dengan baik dan aplikasi Streamlit berfungsi sesuai harapan. 
	• 	Langkah:  
1.	Pastikan semua file (konfigurasi.py, database.py, model.py, manajer_anggaran.py, streamlit_app.py) berada dalam satu direktori. 
2.	Pastikan 	database pengeluaran_harian.db 	sudah 	dibuat 	(jalankan setup_db_pengeluaran.py jika belum). 
3.	Buka terminal di direktori tersebut. 
4.	Jalankan aplikasi: streamlit run streamlit_app.py 
5.	Lakukan pengujian menyeluruh seperti pada Praktikum 6 di versi non-modular sebelumnya:  
▪	Uji tambah data (valid dan invalid). 
▪	Uji lihat riwayat (pastikan data baru muncul setelah refresh/rerun). 
▪	Uji lihat total (pastikan terupdate). 
▪	Uji lihat ringkasan kategori (tabel dan grafik). 
▪	Uji filter tanggal pada ringkasan. 
▪	Periksa apakah ada error di terminal atau di halaman web. 
6.	(Opsional) Periksa kembali isi file database .db. 
Observasi: Aplikasi modular seharusnya berfungsi sama seperti versi gabungan, tetapi struktur kodenya lebih rapi dan terorganisir. Perhatikan pesan inisialisasi AnggaranHarian yang muncul di terminal (seharusnya hanya sekali karena cache). 

# F. Penugasan 

1.	Kumpulkan Laporan Praktikum dari jobsheet ini dalam bentuk Microsoft word sesuai dengan format jobsheet praktikum dan dikumpulkan di web LMS. (JANGAN DALAM BENTUK PDF) 
2.	Kumpulkan luaran kode praktikum dalam bentuk ipynb yang sudah diunggah pada akun github masing-masing. Lampirkan tautan github yang sudah di unggah melalui laman LMS. 
3.	Tambahkan kode Program fungsionalitas Hapus Transaksi yang sudah dibuat. Langkah-langkah Pengembangan: 
    1) Backend (manajer_anggaran.py):  
•	Tambahkan metode baru hapus_transaksi(self, id_transaksi: int) -> bool pada kelas AnggaranHarian. 
•	Metode ini harus menggunakan fungsi database.execute_query() untuk menjalankan perintah SQL DELETE FROM transaksi WHERE id = ? dengan id_transaksi yang diberikan. 
•	Metode ini mengembalikan True jika penghapusan berhasil (query dieksekusi tanpa error SQLite), False jika gagal. 


In [ ]:
# manajer_anggaran.py
import datetime
import pandas as pd
from model import Transaksi
import database # Impor modul database kita

class AnggaranHarian:
    """Mengelola logika bisnis pengeluaran harian (Repository Pattern)."""
    _db_setup_done = False # Flag untuk memastikan setup DB hanya dicek sekali per sesi

    def __init__(self):
        if not AnggaranHarian._db_setup_done:
            print("[AnggaranHarian] Melakukan pengecekan/setup database awal...")
            if database.setup_database_initial(): # Panggil fungsi setup dari database.py
                AnggaranHarian._db_setup_done = True
                print("[AnggaranHarian] Database siap.")
            else:
                print("[AnggaranHarian] KRITICAL: Setup database awal GAGAL!")

    def tambah_transaksi(self, transaksi: Transaksi) -> bool:
        if not isinstance(transaksi, Transaksi) or transaksi.jumlah <= 0: return False
        sql = "INSERT INTO transaksi (deskripsi, jumlah, kategori, tanggal) VALUES (?, ?, ?, ?)"
        params = (transaksi.deskripsi, transaksi.jumlah, transaksi.kategori, transaksi.tanggal.strftime("%Y-%m-%d"))
        last_id = database.execute_query(sql, params)
        if last_id is not None:
            transaksi.id = last_id
            return True
        return False

    def hapus_transaksi(self, id_transaksi: int) -> bool:
        """
        Menghapus transaksi berdasarkan ID-nya.
        Metode ini menjalankan perintah SQL DELETE menggunakan database.execute_query().
        Mengembalikan True jika berhasil, False jika gagal.
        """
        if not isinstance(id_transaksi, int) or id_transaksi <= 0:
            return False
        sql = "DELETE FROM transaksi WHERE id = ?"
        params = (id_transaksi,)
        try:
            database.execute_query(sql, params)
            print(f"INFO: Transaksi dengan ID {id_transaksi} berhasil dihapus dari database.")
            return True # Berhasil jika tidak ada error
        except Exception as e:
            print(f"ERROR: Gagal menghapus transaksi ID {id_transaksi}: {e}")
            return False

    def get_semua_transaksi_obj(self) -> list[Transaksi]:
        sql = "SELECT id, deskripsi, jumlah, kategori, tanggal FROM transaksi ORDER BY tanggal DESC, id DESC"
        rows = database.fetch_query(sql, fetch_all=True)
        transaksi_list = []
        if rows:
            for row in rows:
                transaksi_list.append(Transaksi(id_transaksi=row['id'], deskripsi=row['deskripsi'], jumlah=row['jumlah'], kategori=row['kategori'], tanggal=row['tanggal']))
        return transaksi_list

    def get_dataframe_transaksi(self, filter_tanggal: datetime.date | None = None) -> pd.DataFrame:
        query = "SELECT id, tanggal, kategori, deskripsi, jumlah FROM transaksi"
        params = None
        if filter_tanggal:
            query += " WHERE tanggal = ?"
            params = (filter_tanggal.strftime("%Y-%m-%d"),)
        query += " ORDER BY tanggal DESC, id DESC"
        df = database.get_dataframe(query, params=params)
        if not df.empty:
              try:
                  import locale
                  locale.setlocale(locale.LC_ALL, 'id_ID.UTF-8')
                  df['Jumlah (Rp)'] = df['jumlah'].map(lambda x: locale.currency(x or 0, grouping=True, symbol='Rp ')[:-3])
              except:
                   df['Jumlah (Rp)'] = df['jumlah'].map(lambda x: f"Rp {x or 0:,.0f}".replace(",", "."))
              df = df[['id', 'tanggal', 'kategori', 'deskripsi', 'Jumlah (Rp)']]
        return df

    def hitung_total_pengeluaran(self, tanggal: datetime.date | None = None) -> float:
        sql = "SELECT SUM(jumlah) FROM transaksi"
        params = None
        if tanggal:
            sql += " WHERE tanggal = ?"
            params = (tanggal.strftime("%Y-%m-%d"),)
        result = database.fetch_query(sql, params=params, fetch_all=False)
        if result and result[0] is not None:
            return float(result[0])
        return 0.0

    def get_pengeluaran_per_kategori(self, tanggal: datetime.date | None = None) -> dict:
        hasil = {}
        sql = "SELECT kategori, SUM(jumlah) FROM transaksi"
        params = []
        if tanggal:
            sql += " WHERE tanggal = ?"
            params.append(tanggal.strftime("%Y-%m-%d"))
        sql += " GROUP BY kategori HAVING SUM(jumlah) > 0 ORDER BY SUM(jumlah) DESC"
        rows = database.fetch_query(sql, params=tuple(params) if params else None, fetch_all=True)
        if rows:
            for row in rows:
                kategori = row['kategori'] if row['kategori'] else "Lainnya"
                jumlah = float(row[1]) if row[1] is not None else 0.0
                hasil[kategori] = jumlah
        return hasil

2) Frontend (streamlit_app.py):  
•	Modifikasi tampilan pada tab "Riwayat Lengkap". Anda perlu cara untuk memilih atau menargetkan transaksi yang ingin dihapus. Beberapa ide (pilih salah satu atau cari cara lain):  
o	Tombol per Baris: Jika memungkinkan dengan st.dataframe atau library tabel lain (streamlit-aggrid), tambahkan kolom baru berisi tombol "Hapus" untuk setiap baris. 
o	Input ID & Tombol Hapus: Tambahkan st.number_input("ID Transaksi Hapus:", min_value=1, step=1) dan tombol st.button("Hapus 
Transaksi Terpilih") di dekat tabel riwayat. 
•	Implementasikan logika saat tombol Hapus ditekan:  
o	Tampilkan konfirmasi kepada pengguna (misalnya menggunakan st.warning dan st.button("Konfirmasi Hapus")). 
o	Jika dikonfirmasi, panggil metode anggaran.hapus_transaksi(id_yang_dipilih). o Tampilkan pesan sukses atau gagal menggunakan st.success atau st.error. o Pastikan data pada tabel riwayat dan ringkasan diperbarui setelah penghapusan 
(gunakan st.cache_data.clear() dan st.rerun()). 


In [ ]:
# main_app.py
import streamlit as st
import datetime
import pandas as pd
import locale

try:
    locale.setlocale(locale.LC_ALL, 'id_ID.UTF-8')
except locale.Error:
    try:
        locale.setlocale(locale.LC_ALL, 'Indonesian_Indonesia.1252')
    except:
        print("Locale id_ID/Indonesian tidak tersedia.")

def format_rp(angka):
    try:
        return locale.currency(angka or 0, grouping=True, symbol='Rp ')[:-3]
    except:
        return f"Rp {angka or 0:,.0f}".replace(",",".")

# --- Impor modul aplikasi ---
try:
    from model import Transaksi
    from manajer_anggaran import AnggaranHarian
    from konfigurasi import KATEGORI_PENGELUARAN
except ImportError as e:
    st.error(f"Gagal mengimpor modul: {e}. Pastikan file .py lain ada di direktori yang sama.")
    st.stop()

st.set_page_config(page_title="Catatan Pengeluaran", layout="wide", initial_sidebar_state="expanded")

@st.cache_resource
def get_anggaran_manager():
    print(">>> STREAMLIT: (Cache Resource) Menginisialisasi AnggaranHarian...")
    return AnggaranHarian()

anggaran = get_anggaran_manager()


def halaman_input(anggaran: AnggaranHarian):
    st.header("💎 Tambah Pengeluaran Baru")
    with st.form("form_transaksi_baru", clear_on_submit=True):
        col1, col2 = st.columns([3, 1])
        with col1:
            deskripsi = st.text_input("Deskripsi*", placeholder="Contoh: Makan siang")
        with col2:
            kategori = st.selectbox("Kategori*:", KATEGORI_PENGELUARAN, index=0)
        col3, col4 = st.columns([1, 1])
        with col3:
            jumlah = st.number_input("Jumlah (Rp)*:", min_value=0.01, step=1000.0, format="%.0f", value=None, placeholder="Contoh: 25000")
        with col4:
            tanggal = st.date_input("Tanggal*:", value=datetime.date.today())
        
        submitted = st.form_submit_button("💾 Simpan Transaksi")
        if submitted:
            if not deskripsi:
                st.warning("Deskripsi wajib diisi!", icon="⚠️")
            elif jumlah is None or jumlah <= 0:
                st.warning("Jumlah harus berupa angka positif!", icon="⚠️")
            else:
                with st.spinner("Menyimpan..."):
                    tx = Transaksi(deskripsi, float(jumlah), kategori, tanggal)
                    if anggaran.tambah_transaksi(tx):
                        st.success("Transaksi berhasil disimpan!", icon="✅")
                        st.cache_data.clear()
                        st.rerun()
                    else:
                        st.error("Gagal menyimpan transaksi.", icon="❌")

def halaman_riwayat(anggaran: AnggaranHarian):
    st.subheader("Detail Semua Transaksi")
    
    if st.button("🔄 Refresh Riwayat"):
        st.cache_data.clear()
        st.rerun()
    
    with st.spinner("Memuat riwayat..."):
        df_transaksi = anggaran.get_dataframe_transaksi()
        
    if df_transaksi is None:
        st.error("Gagal mengambil data riwayat.")
    elif df_transaksi.empty:
        st.info("Belum ada transaksi yang tercatat.")
    else:
        st.dataframe(df_transaksi, use_container_width=True, hide_index=True)

        # --- Fungsionalitas Hapus Transaksi ---
        st.divider()
        st.subheader("🗑️ Hapus Transaksi")
        
        with st.form("form_hapus", clear_on_submit=True):
            id_to_delete = st.number_input("Masukkan ID Transaksi yang akan dihapus:", min_value=1, step=1, value=None)
            submitted_delete = st.form_submit_button("Hapus Transaksi", type="primary")

        if submitted_delete:
            if id_to_delete:
                # Simpan ID yang akan dihapus ke session state untuk proses konfirmasi
                st.session_state['id_to_delete'] = id_to_delete
                st.rerun() # Rerun untuk menampilkan dialog konfirmasi
            else:
                st.warning("Mohon masukkan ID transaksi yang valid.", icon="⚠️")

        # Dialog Konfirmasi Penghapusan
        if 'id_to_delete' in st.session_state and st.session_state['id_to_delete'] is not None:
            id_transaksi = st.session_state['id_to_delete']
            st.warning(f"Anda yakin ingin menghapus transaksi dengan ID **{id_transaksi}**? Tindakan ini tidak dapat dibatalkan.", icon="⚠️")
            
            col1, col2, _ = st.columns([1, 1, 4])
            with col1:
                if st.button("✅ Ya, Konfirmasi Hapus", type="primary"):
                    with st.spinner("Menghapus..."):
                        if anggaran.hapus_transaksi(id_transaksi):
                            st.success(f"Transaksi ID {id_transaksi} berhasil dihapus.")
                        else:
                            st.error(f"Gagal menghapus transaksi ID {id_transaksi}. Mungkin ID tidak ditemukan.")
                    
                    # Bersihkan state dan cache, lalu muat ulang data
                    del st.session_state['id_to_delete']
                    st.cache_data.clear()
                    st.rerun()
            
            with col2:
                if st.button("❌ Batal"):
                    del st.session_state['id_to_delete']
                    st.rerun()

def halaman_ringkasan(anggaran: AnggaranHarian):
    st.subheader("Ringkasan Pengeluaran")
    
    col_filter1, col_filter2 = st.columns([1, 2])
    with col_filter1:
        pilihan_periode = st.selectbox("Filter Periode:", ["Semua Waktu", "Hari Ini", "Pilih Tanggal"], key="filter_periode", on_change=lambda: st.cache_data.clear())

    tanggal_filter = None
    label_periode = "(Semua Waktu)"
    if pilihan_periode == "Hari Ini":
        tanggal_filter = datetime.date.today()
        label_periode = f"({tanggal_filter.strftime('%d %b')})"
    elif pilihan_periode == "Pilih Tanggal":
        if 'tanggal_pilihan_state' not in st.session_state:
            st.session_state.tanggal_pilihan_state = datetime.date.today()
        tanggal_filter = st.date_input("Pilih Tanggal:", value=st.session_state.tanggal_pilihan_state, key="tanggal_pilihan", on_change=lambda: (setattr(st.session_state, 'tanggal_pilihan_state', st.session_state.tanggal_pilihan), st.cache_data.clear()))
        label_periode = f"({tanggal_filter.strftime('%d %b %Y')})"

    with col_filter2:
        @st.cache_data(ttl=300)
        def hitung_total_cached(tgl_filter):
            return anggaran.hitung_total_pengeluaran(tanggal=tgl_filter)
        total_pengeluaran = hitung_total_cached(tanggal_filter)
        st.metric(label=f"Total Pengeluaran {label_periode}", value=format_rp(total_pengeluaran))
    
    st.divider()
    st.subheader(f"Pengeluaran per Kategori {label_periode}")
    @st.cache_data(ttl=300)
    def get_kategori_cached(tgl_filter):
        return anggaran.get_pengeluaran_per_kategori(tanggal=tgl_filter)
    
    with st.spinner(f"Memuat ringkasan kategori..."):
        dict_per_kategori = get_kategori_cached(tanggal_filter)

    if not dict_per_kategori:
        st.info(f"Tidak ada data pengeluaran untuk periode ini.")
    else:
        try:
            data_kategori = [{"Kategori": kat, "Total": jml} for kat, jml in dict_per_kategori.items()]
            df_kategori = pd.DataFrame(data_kategori).sort_values(by="Total", ascending=False).reset_index(drop=True)
            df_kategori['Total (Rp)'] = df_kategori['Total'].apply(format_rp)
            
            col_kat1, col_kat2 = st.columns(2)
            with col_kat1:
                st.write("Tabel Rincian:")
                st.dataframe(df_kategori[['Kategori', 'Total (Rp)']], hide_index=True, use_container_width=True)
            with col_kat2:
                st.write("Grafik:")
                st.bar_chart(df_kategori.set_index('Kategori')['Total'], use_container_width=True)
        except Exception as e:
            st.error(f"Gagal menampilkan ringkasan kategori: {e}")

# --- kode utama streamlit---
def main():
    st.sidebar.title("💰 Catatan Pengeluaran")
    menu_pilihan = st.sidebar.radio("Pilih Menu:", ["Tambah", "Riwayat", "Ringkasan"], key="menu_utama")
    st.sidebar.markdown("---")
    st.sidebar.info("Jobsheet - Aplikasi Keuangan")

    manajer_anggaran = get_anggaran_manager()
    if menu_pilihan == "Tambah":
        halaman_input(manajer_anggaran)
    elif menu_pilihan == "Riwayat":
        halaman_riwayat(manajer_anggaran)
    elif menu_pilihan == "Ringkasan":
        halaman_ringkasan(manajer_anggaran)
        
    st.markdown("---")
    st.caption("Pengembangan Aplikasi Berbasis OOP")

if __name__ == "__main__":
    main()

# G. Kesimpulan 
Jobsheet ini telah membawa mahasiswa pada pengalaman mengintegrasikan berbagai teknologi dan konsep pemrograman modern untuk membangun aplikasi yang cukup kompleks, yaitu sistem presensi berbasis deteksi wajah. Melalui perancangan kelas-kelas OOP, mahasiswa belajar menstrukturkan komponen sistem seperti data pengguna, manajemen database (dengan pola Singleton), dan logika pemrosesan utama. Penggunaan library eksternal seperti MediaPipe untuk computer vision (deteksi wajah) dan sqlite3 untuk persistensi data menunjukkan bagaimana OOP dapat menjadi perekat yang mengelola interaksi antar modul berbeda. Meskipun aspek pengenalan wajah disimulasikan, project ini memberikan fondasi pemahaman tentang alur kerja sistem biometrik sederhana dan tantangan dalam mengintegrasikan input real-time (webcam), pemrosesan data (MediaPipe), penyimpanan data (SQLite), dan penanganan kesalahan (exception handling, logging) dalam satu kerangka kerja OOP yang kohesif dan terstruktur. 
 
